In [ ]:
import tkinter as tk
import time
from datetime import datetime
from tkinter import ttk
import pymongo
from functools import lru_cache
import mysql.connector


class TwitterGUI:
    
    def __init__(self, master):
        
        self.master = master
        self.LRUCache = {}
        master.title("Twitter Data Extractor")
        master.geometry("1920x1080")
        master.config(bg="#FFFFFF")
    
        
        self.query_time_label = tk.Label(self.master, text="")
        self.query_time_label.pack(side="top", anchor="ne", padx=10, pady=10)

        self.hashtag_label = tk.Label(master, text="Enter a hashtag:")
        self.hashtag_label.pack(side="top")
        
        self.hashtag_entry = tk.Entry(master, width=30)
        self.hashtag_entry.pack()
        
        self.from_date_label = tk.Label(master, text="From Date (DD):")
        self.from_date_label.pack()

        self.from_date_entry = tk.Entry(master, width=30)
        self.from_date_entry.pack()

        self.from_month_label = tk.Label(master, text="From Month (MM):")
        self.from_month_label.pack()

        self.from_month_entry = tk.Entry(master, width=30)
        self.from_month_entry.pack()

        self.to_date_label = tk.Label(master, text="To Date (DD):")
        self.to_date_label.pack()

        self.to_date_entry = tk.Entry(master, width=30)
        self.to_date_entry.pack()

        self.to_month_label = tk.Label(master, text="To Month (MM):")
        self.to_month_label.pack()

        self.to_month_entry = tk.Entry(master, width=30)
        self.to_month_entry.pack()


        self.tweet_var = tk.IntVar()
        self.tweet_checkbutton = tk.Checkbutton(master, text="Extract from tweet table", variable=self.tweet_var)
        self.tweet_checkbutton.pack()

        self.retweet_var = tk.IntVar()
        self.retweet_checkbutton = tk.Checkbutton(master, text="Extract from retweet table", variable=self.retweet_var)
        self.retweet_checkbutton.pack()

        self.search_button = tk.Button(master, text="Search", command=self.search_button_clicked)
        self.search_button.pack(pady=10)

        self.table_frame = tk.Frame(master, bg="#FFFFFF", bd=2, relief="groove")
        self.table_frame.pack(side="left", fill="both", expand=True, padx=10, pady=10)
        self.table_frame.pack_propagate(0)

        self.table_label = tk.Label(self.table_frame, text="Twitter Data")
        self.table_label.pack(pady=3)

        self.treeview = ttk.Treeview(self.table_frame, columns=("tweet_id", "user_id", "reply_count", "retweet_count", "fav_count", "text"))
        self.treeview.column("#0", width=5) # Index column
        self.treeview.column("tweet_id", width=100) # Tweet ID column
        self.treeview.column("user_id", width=100) # User ID column
        self.treeview.column("reply_count", width=10) # Reply Count column
        self.treeview.column("retweet_count", width=10) # Retweet Count column
        self.treeview.column("fav_count", width=10) # Favorite Count column
        self.treeview.column("text", width=250) # Favorite Count column
        self.treeview.heading("#0", text="Index")
        self.treeview.heading("tweet_id", text="Tweet ID")
        self.treeview.heading("user_id", text="User ID")
        self.treeview.heading("reply_count", text="friends Count", command=lambda: self.treeview_sort_column("reply_count", False))
        self.treeview.heading("retweet_count", text="Followers Count", command=lambda: self.treeview_sort_column("retweet_count", False))
        self.treeview.heading("fav_count", text="Favorites Count", command=lambda: self.treeview_sort_column("fav_count", False))
        self.treeview.heading("text", text="Text")
        self.treeview.pack(fill="both", expand=True)

        self.treeview.bind("<<TreeviewSelect>>", self.on_treeview_select)
        
    def search_button_clicked(self):
        input_str = self.hashtag_entry.get().strip()
        if input_str.startswith('#'):
            self.get_data_hashtag(input_str[1:])
        elif input_str.startswith('@'):
            self.get_data_screen_name(input_str[1:])
        else:
            self.get_data_string(input_str)
    
    def get_data_hashtag(self, hashtag_passed):
        hashtag = hashtag_passed
        is_tweet = self.tweet_var.get() == 1
        is_retweet = self.retweet_var.get() == 1

        for row in self.treeview.get_children():
            self.treeview.delete(row)

        # Check if the data is in the cache
        cache_key = (hashtag, is_tweet, is_retweet)
        if cache_key in self.cache:
            query_start_time = datetime.now()
            data = self.cache[cache_key]
            query_end_time = datetime.now()
            query_time = query_end_time - query_start_time
            self.query_time_label.config(text="Query Time: {} seconds (from cache)".format(query_time))
            for i, row in enumerate(data):
                self.treeview.insert("", "end", text=i+1, values=row)
        else:
            # Fetch data from the selected table
            if is_tweet:
                table = "tweet"
            elif is_retweet:
                table = "retweet"
            else:
                return            

            cnx = mysql.connector.connect(user='root', password='Nkiranreddy@4',
                                  host='localhost', database='twitter40')
            cursor = cnx.cursor()

            query_start_time = datetime.now()
            query = f"""
                        SELECT {table}.tweet_id, {table}.user_id, user.friends_count, user.followers_count, user.favourites_count
                        FROM {table}
                        JOIN tweet_hashtag ON {table}.tweet_id = tweet_hashtag.tweet_id
                        JOIN hashtag ON tweet_hashtag.hashtag_id = hashtag.hashtag_id
                        JOIN user ON {table}.user_id = user.user_id
                        WHERE hashtag.name = '{hashtag}'
                    """
            cursor.execute(query)
            data = cursor.fetchall()

            query_end_time = datetime.now()
            query_time = query_end_time - query_start_time

            self.query_time_label.config(text="Query Time: {} seconds".format(query_time.total_seconds()))

            # Store the data in the cache
            mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
            mongo_db = mongo_client["mydatabase"]
            mongo_data_dict = {}
            if is_tweet:
                mongo_collection = mongo_db["tweets"]
            else:
                mongo_collection = mongo_db["retweets"]
            mongo_query = {"id": {"$in": [int(row[0]) for row in data]}}
            mongo_data = mongo_collection.find(mongo_query, {"id": 1, "text": 1})
            for record in mongo_data:
                mongo_data_dict[record["id"]] = record
            data_with_text = []
            for row in data:
                tweet_id = int(row[0])
                mongo_record = mongo_data_dict.get(tweet_id)
                if mongo_record:
                    row += (mongo_record.get("text", ""),)
                data_with_text.append(row)
            self.cache[cache_key] = data_with_text

            cursor.close()
            cnx.close()

            for i, row in enumerate(data_with_text):
                self.treeview.insert("", "end", text=i+1, values=row)
                
                
    def get_data_string(self, string_passed):
        search_str = string_passed
        is_tweet = self.tweet_var.get() == 1
        is_retweet = self.retweet_var.get() == 1

        for row in self.treeview.get_children():
            self.treeview.delete(row)

        # Check if the data is in the cache
        cache_key_1 = (search_str, is_tweet, is_retweet)
        if cache_key_1 in self.cache_1:
            query_start_time = datetime.now()
            data = self.cache_1[cache_key_1]
            query_end_time = datetime.now()
            query_time = query_end_time - query_start_time
            self.query_time_label.config(text="Query Time: {} seconds (from cache)".format(query_time))
            for i, row in enumerate(data):
                self.treeview.insert("", "end", text=i+1, values=row)
        else:
            # Search for data in MongoDB
            mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
            mongo_db = mongo_client["mydatabase"]
            mongo_data_dict = {}
            if is_tweet:
                mongo_collection = mongo_db["tweets"]
            else:
                mongo_collection = mongo_db["retweets"]
            mongo_query = {"text": {"$regex": search_str}}
            mongo_data = mongo_collection.find(mongo_query, {"id": 1, "text": 1})
            tweet_ids = [record["id"] for record in mongo_data]

            # Fetch data from the selected table in MySQL
            if is_tweet:
                table = "tweet"
            elif is_retweet:
                table = "retweet"
            else:
                return

            cnx = mysql.connector.connect(user='root', password='Nkiranreddy@4',
                                  host='localhost', database='twitter40')
            cursor = cnx.cursor()

            query_start_time = datetime.now()
            query = f"""
                        SELECT {table}.tweet_id, {table}.user_id, user.friends_count, user.followers_count, user.favourites_count
                        FROM {table}
                        JOIN tweet_hashtag ON {table}.tweet_id = tweet_hashtag.tweet_id
                        JOIN hashtag ON tweet_hashtag.hashtag_id = hashtag.hashtag_id
                        JOIN user ON {table}.user_id = user.user_id
                        WHERE hashtag.name = '{search_str}' OR {table}.tweet_id IN ({','.join(map(str, tweet_ids))})
                    """
            
            cursor.execute(query)
            data = cursor.fetchall()

            query_end_time = datetime.now()
            query_time = query_end_time - query_start_time

            self.query_time_label.config(text="Query Time: {} seconds".format(query_time.total_seconds()))

            # Store the data in the cache
            mongo_query = {"id": {"$in": tweet_ids}}
            mongo_data = mongo_collection.find(mongo_query, {"id": 1, "text": 1})
            for record in mongo_data:
                mongo_data_dict[record["id"]] = record
            data_with_text = []
            for row in data:
                tweet_id = int(row[0])
                mongo_record = mongo_data_dict.get(tweet_id)
                if mongo_record:
                    row += (mongo_record.get("text", ""),)
                data_with_text.append(row)
            self.cache_1[cache_key_1] = data_with_text

            cursor.close()
            cnx.close()

            for i, row in enumerate(data_with_text):
                self.treeview.insert("", "end", text=i+1, values=row)

    def get_data_screen_name(self, screen_name_passed):
        screen_name = screen_name_passed
        is_tweet = self.tweet_var.get() == 1
        is_retweet = self.retweet_var.get() == 1

        for row in self.treeview.get_children():
            self.treeview.delete(row)

        # Check if the data is in the cache
        cache_key_3 = (screen_name, is_tweet, is_retweet)
        if cache_key_3 in self.cache_3:
            query_start_time = datetime.now()
            data = self.cache_3[cache_key_3]
            query_end_time = datetime.now()
            query_time = query_end_time - query_start_time
            self.query_time_label.config(text="Query Time: {} seconds (from cache)".format(query_time))
            for i, row in enumerate(data):
                self.treeview.insert("", "end", text=i+1, values=row)
        else:
            # Fetch data from the selected table
            if is_tweet:
                table = "tweet"
            elif is_retweet:
                table = "retweet"
            else:
                return            

            cnx = mysql.connector.connect(user='root', password='Nkiranreddy@4',
                                  host='localhost', database='twitter40')
            cursor = cnx.cursor()

            query_start_time = datetime.now()
            query = f"""
                        SELECT {table}.tweet_id, {table}.user_id, user.friends_count, user.followers_count, user.favourites_count
                        FROM {table}
                        JOIN user ON {table}.user_id = user.user_id
                        WHERE user.screen_name LIKE '%{screen_name}%'
                    """
            cursor.execute(query)
            data = cursor.fetchall()

            query_end_time = datetime.now()
            query_time = query_end_time - query_start_time

            self.query_time_label.config(text="Query Time: {} seconds".format(query_time.total_seconds()))

            # Store the data in the cache
            mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
            mongo_db = mongo_client["mydatabase"]
            mongo_data_dict = {}
            if is_tweet:
                mongo_collection = mongo_db["tweets"]
            elif is_retweet:
                mongo_collection = mongo_db["retweets"]
            else:
                return  
            mongo_query = {"id": {"$in": [int(row[0]) for row in data]}}
            mongo_data = mongo_collection.find(mongo_query, {"id": 1, "text": 1})
            for record in mongo_data:
                mongo_data_dict[record["id"]] = record
            data_with_text = []
            for row in data:
                tweet_id = int(row[0])
                mongo_record = mongo_data_dict.get(tweet_id)
                if mongo_record:
                    row += (mongo_record.get("text", ""),)
                data_with_text.append(row)
            self.cache_3[cache_key_3] = data_with_text

            cursor.close()
            cnx.close()

            for i, row in enumerate(data_with_text):
                self.treeview.insert("", "end", text=i+1, values=row)



        
    def on_treeview_select(self, event):
        # Get the selected row from the treeview
        selection = self.treeview.selection()
        if not selection:
            return
        item = self.treeview.item(selection[0])
        values = item['values']
        user_id = values[1]

        # Fetch user data from the database using the user ID
        cnx = mysql.connector.connect(user='root', password='Nkiranreddy@4', host='localhost', database='twitter40')
        cursor = cnx.cursor()
        query = f"""
            SELECT screen_name, location, description, followers_count, friends_count
            FROM user
            WHERE user_id = '{user_id}'
        """
        cursor.execute(query)
        user_data = cursor.fetchone()

        cursor.close()
        cnx.close()

        # Create a new window to display the user data
        self.new_window = tk.Toplevel(self.master)
        self.new_window.title("User Data")
        self.new_window.geometry("1920x1080") 

        # Add a treeview widget to the new window and configure its columns
        self.treeview2 = ttk.Treeview(self.new_window, columns=("Attribute"))
        self.treeview2.column("Attribute", width=800) # Tweet ID column
        self.treeview2.heading("Attribute", text="Attributes")
        self.treeview2.pack(fill="both", expand=True)

        
        x = int(values[0])
        mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
        mongo_db = mongo_client["mydatabase"]
        
        is_tweet = self.tweet_var.get() == 1
        is_retweet = self.retweet_var.get() == 1
        
        if is_tweet:
                mongo_collection = mongo_db["tweets"]
        else:
            mongo_collection = mongo_db["retweets"]
            
        mongo_query = {"id": x}
        mongo_data = mongo_collection.find_one(mongo_query)
        text = mongo_data.get("text", "")
        
        # Insert user data into the treeview
        self.treeview2.insert("", tk.END, text="Screen Name", values=(user_data[0]), tags=('bold',))
        self.treeview2.insert("", tk.END, text="Location", values=(user_data[1] if user_data[1] is not None else ""), tags=('bold',))
        self.treeview2.insert("", tk.END, text="Description", values=(user_data[2]), tags=('bold',))
        self.treeview2.insert("", tk.END, text="Followers Count", values=(user_data[3]), tags=('bold',))
        self.treeview2.insert("", tk.END, text="Friends Count", values=(user_data[4]), tags=('bold',))
        self.treeview2.insert("", tk.END, text="Text", values=(text,), tags=('bold',))

        # Set the tag configuration for bold text
        self.treeview2.tag_configure('bold', font=('Arial', 10, 'bold'))


        
    def add_text_line_by_line(self, text):
        MAX_LINE_LENGTH = 50
        while len(text) > MAX_LINE_LENGTH:
            self.right_listbox.insert(tk.END, text[:MAX_LINE_LENGTH])
            text = text[MAX_LINE_LENGTH:]
        self.right_listbox.insert(tk.END, text)

        
        
    def treeview_sort_column(self, col, reverse):
        items = [(self.treeview.set(k, col), k) for k in self.treeview.get_children('')]
        items.sort(reverse=reverse)

        # Rearrange the items in sorted positions
        for index, (val, k) in enumerate(items):
            self.treeview.move(k, '', index)

        # Reverse sort direction for next time
        self.treeview.heading(col, command=lambda: self.treeview_sort_column(col, not reverse))

        

root = tk.Tk()
twitter_gui = TwitterGUI(root)
root.mainloop()

